Credit for this script:
- stackoverlflow alias: SirThanksALot and contributors
- Apr 24 '20
- Python image scraping
- https://stackoverflow.com/questions/61406950/python-image-scraping

In [1]:
from selenium import webdriver
import os
import time
import requests
from selenium.webdriver.common.by import By

In [2]:
def scroll_to_end(wd):
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(sleep_between_interactions)

In [3]:
def fetch_image_urls(query: str, max_links_to_fetch: int, wd: webdriver, sleep_between_interactions: int = 1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

        # build the google query

    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)

        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                return image_urls
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
#             return load_more_button
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [4]:
def persist_image(folder_path:str,url:str, counter):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb')
        f.write(image_content)
        f.close()
        print(f"SUCCESS - saved {url} - as {folder_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [5]:
def search_and_download(search_term: str, driver_path: str, target_path='./images', number_images=10):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=1)

    counter = 0
    for elem in res:
        persist_image(target_folder, elem, counter)
        counter += 1

DRIVER_PATH = './chromedriver'
search_term = 'cats'
# num of images you can pass it from here  by default it's 10 if you are not passing
number_images = 500
search_and_download(search_term=search_term, driver_path=DRIVER_PATH,number_images = number_images)

<ipython-input-5-ba9f0aecbfac>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
<ipython-input-3-417602b6418c>:20: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


<ipython-input-3-417602b6418c>:34: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 103 image links, looking for more ...


<ipython-input-3-417602b6418c>:48: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  load_more_button = wd.find_element_by_css_selector(".mye4qd")


...

SUCCESS - saved https://www.cats.org.uk/media/9603/photo-1559624989-7b9303bd9792.jpg?width=500 - as ./images\cats
SUCCESS - saved https://cdn.britannica.com/q:60/91/181391-050-1DA18304/cat-toes-paw-number-paws-tiger-tabby.jpg - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSXIUEW0-mJjpRHFIp0CHmzNlkWQlFelvm2oA&usqp=CAU - as ./images\cats
SUCCESS - saved https://static.wikia.nocookie.net/minecraft_gamepedia/images/d/d3/Siamese_Cat_JE3.png/revision/latest?cb=20210317194731 - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRGTuuO-QkQ2u9OBcx1EJvxa4jJhavcjsSBLg&usqp=CAU - as ./images\cats
SUCCESS - saved https://cdn.vox-cdn.com/thumbor/FjEAPZ8kHM8Wh9VT4CbIj7kUpSs=/1400x1400/filters:format(jpeg)/cdn.vox-cdn.com/uploads/chorus_asset/file/18313281/cats_alamy_ECRAF0.jpg - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRjR65-nilIKLKMa7VghI95LzQnhhXI719XLw&usqp=CAU - as ./images\

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQE_n4nkqm_zEH2Z4cr4ucW4iFjdtfK1rI3zw&usqp=CAU - as ./images\cats
SUCCESS - saved https://i.ytimg.com/vi/lZqqMtW87hU/maxresdefault.jpg - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR4AGR46LbJYplf_PpUlkCqJpYxvMvT1-tfjA&usqp=CAU - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQwerk0thymW__xc29Qd3Wu6HIFhmVXxft2FQ&usqp=CAU - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSwMq5lzTOPxX7sLQI2DRw6pT03wx25H4TKxA&usqp=CAU - as ./images\cats
SUCCESS - saved http://www.alleycat.org/wp-content/uploads/2015/04/Stray_Feral_Body_Language_Image.png - as ./images\cats
SUCCESS - saved https://jngnposwzs-flywheel.netdna-ssl.com/wp-content/uploads/2019/05/Transparent-OrangeWhiteCat-764x1024.png - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSWFd6qU4m3za4LKn0pdd9Rw13_opsEmVxM_

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQl5TaexXMOe8wGn-RkRmdT7q7Fia_3Z8k10w&usqp=CAU - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQrjcAh2Pw6hbub-OsL0a8ogFEALn-e5SCylg&usqp=CAU - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRVsgRZJarJn83qU2VdwLS0kFEf4UF1T6gAdg&usqp=CAU - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTuc_0rADWnALoaKzs_uLFc3R0GT55G5v2ozw&usqp=CAU - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQwnlTHbll8_2sD9k5AkrYh5U-M3jICHkIh5g&usqp=CAU - as ./images\cats
SUCCESS - saved https://cdn.theatlantic.com/thumbor/tK3KeCj08mDDu431jVj3ex_GqXc=/0x699:6720x4479/960x540/media/img/mt/2021/05/GettyImages_1218380632/original.jpg - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSd6ONLDvyHJnBoD_-tER17XoFI1SEEqlOZ2A&usqp=CAU - as ./images\cats
SUCCESS -

SUCCESS - saved https://www.funkidslive.com/wp-content/uploads/2020/12/Top-10-Facts-Cats-1200x675.jpg - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRUVAMQMPtC_qw992Cp_NCfZR-mgTAFggxKLA&usqp=CAU - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSbwkR1kfLDDpOnRXZ85UnRk9evjd6VakzYLA&usqp=CAU - as ./images\cats
SUCCESS - saved https://www.goodnewsnetwork.org/wp-content/uploads/2021/07/cat-people-official-poster-netflix.jpg - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSpONrZP4oaflnWAwgvgS7eI8761po9Zg0Z2A&usqp=CAU - as ./images\cats
SUCCESS - saved https://freshpet.com/wp-content/uploads/2019/09/cat-2224201_1280.jpg - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTllq2982JoUTC7Hks-OIjJifR0AgNmOdPaBA&usqp=CAU - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQxsT5h24NFxfRIjjJMAecnzDOA3KUzRM

SUCCESS - saved https://ychef.files.bbci.co.uk/976x549/p09b2ctv.jpg - as ./images\cats
SUCCESS - saved http://images.contentful.com/e43mbx7oxv8s/dTeQVZbR72uiYkEAYKyUS/bd3825a93aace2c7b368a7a742b1000f/000122_ZV_SCSW_HeroImages_1900x1024_D1.jpg - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTid_aFG_c6-ydj7aaSYAF0U4s9tU7nzU-Giw&usqp=CAU - as ./images\cats
SUCCESS - saved https://cdn.hswstatic.com/gif/munchkin-cat.jpg - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRDsha7wO-Y1OqEuKkfio4Ao2QlMm1EfEe0kw&usqp=CAU - as ./images\cats
SUCCESS - saved https://wildcatconservation.org/wp-content/uploads/2020/06/Young-adult-un-collared-male2.jpg - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSjuNbF4c7CXKlRAe8036w_6Xa3EzShbUnSOQ&usqp=CAU - as ./images\cats
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT8U8zT1oE7Kr31jTCHZMMzZ2wgCP5Ckkiaiw&usqp=CAU - as ./